In [1]:
import sqlhelpers as sh

In [2]:
# %load Exercise Files\\Chapters\\sql-chap11.txt
-- 01 simple subselect
-- world.db

CREATE TABLE t ( a TEXT, b TEXT );
INSERT INTO t VALUES ( 'NY0123', 'US4567' );
INSERT INTO t VALUES ( 'AZ9437', 'GB1234' );
INSERT INTO t VALUES ( 'CA1279', 'FR5678' );
SELECT * FROM t;

SELECT SUBSTR(a, 1, 2) AS State, SUBSTR(a, 3) AS SCode, 
  SUBSTR(b, 1, 2) AS Country, SUBSTR(b, 3) AS CCode FROM t;

SELECT co.Name, ss.CCode FROM (
    SELECT SUBSTR(a, 1, 2) AS State, SUBSTR(a, 3) AS SCode,
      SUBSTR(b, 1, 2) AS Country, SUBSTR(b, 3) AS CCode FROM t
  ) AS ss
  JOIN Country AS co
    ON co.Code2 = ss.Country
;

DROP TABLE t;

-- 02 searching within a result set
-- album.db

SELECT DISTINCT album_id FROM track WHERE duration <= 90;

SELECT * FROM album
  WHERE id IN (SELECT DISTINCT album_id FROM track WHERE duration <= 90)
;

SELECT a.title AS album, a.artist, t.track_number AS seq, t.title, t.duration AS secs
  FROM album AS a
  JOIN track AS t
    ON t.album_id = a.id
  WHERE a.id IN (SELECT DISTINCT album_id FROM track WHERE duration <= 90)
  ORDER BY a.title, t.track_number
;

SELECT a.title AS album, a.artist, t.track_number AS seq, t.title, t.duration AS secs
  FROM album AS a
  JOIN (
    SELECT album_id, track_number, duration, title
      FROM track
      WHERE duration <= 90
  ) AS t
    ON t.album_id = a.id
  ORDER BY a.title, t.track_number
;

-- 03 Creating a view
-- album.db

SELECT id, album_id, title, track_number, duration / 60 AS m, duration % 60 AS s FROM track;

CREATE VIEW trackView AS
  SELECT id, album_id, title, track_number, 
    duration / 60 AS m, duration % 60 AS s FROM track;
SELECT * FROM trackView;

SELECT a.title AS album, a.artist, t.track_number AS seq, t.title, t.m, t.s
  FROM album AS a
  JOIN trackView AS t
    ON t.album_id = a.id
  ORDER BY a.title, t.track_number
;

SELECT a.title AS album, a.artist, t.track_number AS seq, t.title, 
    t.m || ':' || substr('00' || t.s, -2, 2) AS dur
  FROM album AS a
  JOIN trackView AS t
    ON t.album_id = a.id
  ORDER BY a.title, t.track_number
;

DROP VIEW IF EXISTS trackView;

-- 04 Joined view
-- album.db

SELECT a.artist AS artist,
    a.title AS album,
    t.title AS track,
    t.track_number AS trackno,
    t.duration / 60 AS m,
    t.duration % 60 AS s
  FROM track AS t
    JOIN album AS a
      ON a.id = t.album_id
;

CREATE VIEW joinedAlbum AS
  SELECT a.artist AS artist,
      a.title AS album,
      t.title AS track,
      t.track_number AS trackno,
      t.duration / 60 AS m,
      t.duration % 60 AS s
    FROM track AS t
    JOIN album AS a
      ON a.id = t.album_id
;

SELECT * FROM joinedAlbum;
SELECT * FROM joinedAlbum WHERE artist = 'Jimi Hendrix';

SELECT artist, album, track, trackno, 
   m || ':' || substr('00' || s, -2, 2) AS duration
    FROM joinedAlbum;

DROP VIEW IF EXISTS joinedAlbum;



# Creating a subselect

In [3]:
# set up table

q1 = "CREATE TABLE t ( a TEXT, b TEXT );"
q2 = "INSERT INTO t VALUES ( 'NY0123', 'US4567' );"
q3 = "INSERT INTO t VALUES ( 'AZ9437', 'GB1234' );"
q4 = "INSERT INTO t VALUES ( 'CA1279', 'FR5678' );"

sh.perform_insert_and_stats(q1, 'world.db')
sh.perform_insert_and_stats(q2, 'world.db')
sh.perform_insert_and_stats(q3, 'world.db')
sh.perform_insert_and_stats(q4, 'world.db')

### Query Summary ###

DATABASE QUERIED: world.db
QUERY: ||| CREATE TABLE t ( a TEXT, b TEXT ); |||
### Query Summary ###

DATABASE QUERIED: world.db
QUERY: ||| INSERT INTO t VALUES ( 'NY0123', 'US4567' ); |||
### Query Summary ###

DATABASE QUERIED: world.db
QUERY: ||| INSERT INTO t VALUES ( 'AZ9437', 'GB1234' ); |||
### Query Summary ###

DATABASE QUERIED: world.db
QUERY: ||| INSERT INTO t VALUES ( 'CA1279', 'FR5678' ); |||


In [4]:
# see what we have
q = "SELECT * FROM t;"
sh.query_result_stats(q, "world.db")

### Query Summary ###

DATABASE QUERIED: world.db
QUERY: ||| SELECT * FROM t; |||

There are 3 rows in the query results table
There are 2 columns in the query results table

First 5 rows of the query results:
        a       b
0  NY0123  US4567
1  AZ9437  GB1234
2  CA1279  FR5678


In [5]:
q = """
SELECT SUBSTR(a, 1, 2) AS State, SUBSTR(a, 3) AS SCode, 
  SUBSTR(b, 1, 2) AS Country, SUBSTR(b, 3) AS CCode FROM t;
"""
sh.query_result_stats(q, "world.db")

### Query Summary ###

DATABASE QUERIED: world.db
QUERY: ||| 
SELECT SUBSTR(a, 1, 2) AS State, SUBSTR(a, 3) AS SCode, 
  SUBSTR(b, 1, 2) AS Country, SUBSTR(b, 3) AS CCode FROM t;
 |||

There are 3 rows in the query results table
There are 4 columns in the query results table

First 5 rows of the query results:
  State SCode Country CCode
0    NY  0123      US  4567
1    AZ  9437      GB  1234
2    CA  1279      FR  5678


In [6]:
q = """
SELECT co.Name, ss.CCode FROM (
    SELECT SUBSTR(a, 1, 2) AS State, SUBSTR(a, 3) AS SCode,
      SUBSTR(b, 1, 2) AS Country, SUBSTR(b, 3) AS CCode FROM t
  ) AS ss
  JOIN Country AS co
    ON co.Code2 = ss.Country
;
"""
sh.query_result_stats(q, "world.db")

### Query Summary ###

DATABASE QUERIED: world.db
QUERY: ||| 
SELECT co.Name, ss.CCode FROM (
    SELECT SUBSTR(a, 1, 2) AS State, SUBSTR(a, 3) AS SCode,
      SUBSTR(b, 1, 2) AS Country, SUBSTR(b, 3) AS CCode FROM t
  ) AS ss
  JOIN Country AS co
    ON co.Code2 = ss.Country
;
 |||

There are 3 rows in the query results table
There are 2 columns in the query results table

First 5 rows of the query results:
             Name CCode
0   United States  4567
1  United Kingdom  1234
2          France  5678


In [7]:
q = "DROP TABLE t;"
sh.perform_insert_and_stats(q, 'world.db')

### Query Summary ###

DATABASE QUERIED: world.db
QUERY: ||| DROP TABLE t; |||


# Searching within a result set

In [8]:
q = "SELECT DISTINCT album_id FROM track WHERE duration <= 90;"
sh.query_result_stats(q, "album.db")

### Query Summary ###

DATABASE QUERIED: album.db
QUERY: ||| SELECT DISTINCT album_id FROM track WHERE duration <= 90; |||

There are 2 rows in the query results table
There are 1 columns in the query results table

First 5 rows of the query results:
   album_id
0        11
1        13


In [9]:
q = """
SELECT * FROM album
  WHERE id IN (SELECT DISTINCT album_id FROM track WHERE duration <= 90)
;
"""
sh.query_result_stats(q, "album.db")

### Query Summary ###

DATABASE QUERIED: album.db
QUERY: ||| 
SELECT * FROM album
  WHERE id IN (SELECT DISTINCT album_id FROM track WHERE duration <= 90)
;
 |||

There are 2 rows in the query results table
There are 5 columns in the query results table

First 5 rows of the query results:
   id                title                artist     label    released
0  11  Hendrix in the West          Jimi Hendrix   Polydor  1972-01-00
1  13        Birds of Fire  Mahavishnu Orchestra  Columbia  1973-03-00


# Creating a view

In [10]:
q = "SELECT id, album_id, title, track_number, duration / 60 AS m, duration % 60 AS s FROM track;"
sh.query_result_stats(q, "album.db")

### Query Summary ###

DATABASE QUERIED: album.db
QUERY: ||| SELECT id, album_id, title, track_number, duration / 60 AS m, duration % 60 AS s FROM track; |||

There are 63 rows in the query results table
There are 6 columns in the query results table

First 5 rows of the query results:
   id  album_id                   title  track_number  m   s
0   1         1  Bright Lights Big City             1  5  20
1   2         1              Night Life             2  5  44
2   3         1      Basin Street Blues             5  4  56
3   4         1                Caldonia             3  3  25
4   5         1                Stardust             4  5   8


In [12]:
q = """
CREATE VIEW trackView AS
  SELECT id, album_id, title, track_number, 
    duration / 60 AS m, duration % 60 AS s FROM track;
"""
sh.perform_insert_and_stats(q, 'album.db')

### Query Summary ###

DATABASE QUERIED: album.db
QUERY: ||| 
CREATE VIEW trackView AS
  SELECT id, album_id, title, track_number, 
    duration / 60 AS m, duration % 60 AS s FROM track;
 |||


In [13]:
q = "SELECT * FROM trackView;"
sh.query_result_stats(q, "album.db")

### Query Summary ###

DATABASE QUERIED: album.db
QUERY: ||| SELECT * FROM trackView; |||

There are 63 rows in the query results table
There are 6 columns in the query results table

First 5 rows of the query results:
   id  album_id                   title  track_number  m   s
0   1         1  Bright Lights Big City             1  5  20
1   2         1              Night Life             2  5  44
2   3         1      Basin Street Blues             5  4  56
3   4         1                Caldonia             3  3  25
4   5         1                Stardust             4  5   8


In [14]:
# delete the view
q = "DROP VIEW IF EXISTS trackView;"
sh.perform_insert_and_stats(q, 'album.db')

### Query Summary ###

DATABASE QUERIED: album.db
QUERY: ||| DROP VIEW IF EXISTS trackView; |||


# Creating a joined view

In [15]:
q = """
SELECT a.artist AS artist,
    a.title AS album,
    t.title AS track,
    t.track_number AS trackno,
    t.duration / 60 AS m,
    t.duration % 60 AS s
  FROM track AS t
    JOIN album AS a
      ON a.id = t.album_id
;
"""
sh.query_result_stats(q, "album.db")

### Query Summary ###

DATABASE QUERIED: album.db
QUERY: ||| 
SELECT a.artist AS artist,
    a.title AS album,
    t.title AS track,
    t.track_number AS trackno,
    t.duration / 60 AS m,
    t.duration % 60 AS s
  FROM track AS t
    JOIN album AS a
      ON a.id = t.album_id
;
 |||

There are 63 rows in the query results table
There are 6 columns in the query results table

First 5 rows of the query results:
                              artist                   album  \
0  Willie Nelson and Wynton Marsalis  Two Men with the Blues   
1  Willie Nelson and Wynton Marsalis  Two Men with the Blues   
2  Willie Nelson and Wynton Marsalis  Two Men with the Blues   
3  Willie Nelson and Wynton Marsalis  Two Men with the Blues   
4  Willie Nelson and Wynton Marsalis  Two Men with the Blues   

                    track  trackno  m   s  
0  Bright Lights Big City        1  5  20  
1              Night Life        2  5  44  
2      Basin Street Blues        5  4  56  
3                Caldon

In [16]:
# make a view from the query
q = """
CREATE VIEW joinedAlbum AS
  SELECT a.artist AS artist,
      a.title AS album,
      t.title AS track,
      t.track_number AS trackno,
      t.duration / 60 AS m,
      t.duration % 60 AS s
    FROM track AS t
    JOIN album AS a
      ON a.id = t.album_id
;
"""
sh.perform_insert_and_stats(q, 'album.db')

### Query Summary ###

DATABASE QUERIED: album.db
QUERY: ||| 
CREATE VIEW joinedAlbum AS
  SELECT a.artist AS artist,
      a.title AS album,
      t.title AS track,
      t.track_number AS trackno,
      t.duration / 60 AS m,
      t.duration % 60 AS s
    FROM track AS t
    JOIN album AS a
      ON a.id = t.album_id
;
 |||


In [17]:

q = "SELECT * FROM joinedAlbum;"
sh.query_result_stats(q, "album.db")

### Query Summary ###

DATABASE QUERIED: album.db
QUERY: ||| SELECT * FROM joinedAlbum; |||

There are 63 rows in the query results table
There are 6 columns in the query results table

First 5 rows of the query results:
                              artist                   album  \
0  Willie Nelson and Wynton Marsalis  Two Men with the Blues   
1  Willie Nelson and Wynton Marsalis  Two Men with the Blues   
2  Willie Nelson and Wynton Marsalis  Two Men with the Blues   
3  Willie Nelson and Wynton Marsalis  Two Men with the Blues   
4  Willie Nelson and Wynton Marsalis  Two Men with the Blues   

                    track  trackno  m   s  
0  Bright Lights Big City        1  5  20  
1              Night Life        2  5  44  
2      Basin Street Blues        5  4  56  
3                Caldonia        3  3  25  
4                Stardust        4  5   8  


In [18]:
# clean up
q = "DROP VIEW IF EXISTS joinedAlbum;"
sh.perform_insert_and_stats(q, 'album.db')

### Query Summary ###

DATABASE QUERIED: album.db
QUERY: ||| DROP VIEW IF EXISTS joinedAlbum; |||
